In [ ]:
# Celda 1
# Importamos la función desde nuestro archivo .py
from src.tools.calculadora import calcular_promedio_de_notas

# Celda 2
# Prueba funcional
pregunta1 = "¿cuál es el promedio entre 4.0, 5.5 y 7.0?"
print(calcular_promedio_de_notas(pregunta1))

# Celda 3
# Prueba con errores de tipeo
pregunta2 = "necesito la media de 3,5 y un 2"
print(calcular_promedio_de_notas(pregunta2))

# Celda 4
# Prueba sin números
pregunta3 = "hola cómo estás"
print(calcular_promedio_de_notas(pregunta3))```

*Haz lo mismo en otro notebook para las funciones de `buscador.py`.*

---

### **Paso 4: Crear el Agente (El Cerebro Orquestador)**

Ahora unimos todo en `agent.py`.

```python
# En src/agent.py
import os
from dotenv import load_dotenv
from langchain.agents import tool, AgentExecutor, create_tool_calling_agent
from langchain_core.prompts import ChatPromptTemplate
from langchain_huggingface import HuggingFaceEndpoint

# Importamos nuestras herramientas personalizadas
from src.tools.calculadora import calcular_promedio_de_notas
from src.tools.buscador import buscar_en_faq, buscador_de_reglamentos

# Cargar variables de entorno (el token de Hugging Face)
load_dotenv()

# --- 1. Módulo de Reglas de Seguridad ---
def filtro_de_seguridad(consulta: str) -> bool:
    """
    Filtro simple para detectar consultas inapropiadas.
    Devuelve True si la consulta es segura, False si no lo es.
    """
    consultas_prohibidas = ["examen", "respuestas", "dame la prueba", "hackear"]
    for palabra in consultas_prohibidas:
        if palabra in consulta.lower():
            return False
    return True

# --- 2. Definición de Herramientas para LangChain ---
# LangChain necesita que las funciones estén decoradas con @tool
# La descripción (el docstring) es CRUCIAL. El LLM la usa para decidir qué herramienta usar.

@tool
def calculadora_academica(consulta: str) -> str:
    """
    Útil para calcular el promedio de notas. La entrada debe ser la pregunta completa
    del usuario que contiene las notas.
    """
    return calcular_promedio_de_notas(consulta)

@tool
def buscador_faq(consulta: str) -> str:
    """
    Útil para responder preguntas frecuentes sobre horarios, correos de contacto,
    o procedimientos administrativos simples.
    """
    return buscar_en_faq(consulta)

@tool
def buscador_reglamentos(consulta: str) -> str:
    """
    Útil para responder preguntas específicas sobre el reglamento académico,
    como reglas de asistencia, calificaciones, o condiciones de examen.
    """
    return buscador_de_reglamentos.buscar(consulta)

# --- 3. Creación del Agente ---
def crear_agente():
    """
    Configura y crea el agente conversacional.
    """
    # Lista de herramientas que el agente podrá usar
    tools = [calculadora_academica, buscador_faq, buscador_reglamentos]
    
    # Configuración del LLM (Modelo de Lenguaje) usando Hugging Face
    llm = HuggingFaceEndpoint(
        repo_id="mistralai/Mixtral-8x7B-Instruct-v0.1",
        task="text-generation",
        max_new_tokens=512,
        temperature=0.7,
    )
    
    # El prompt es la instrucción principal que le damos al agente
    prompt = ChatPromptTemplate.from_messages([
        ("system", "Eres un asistente académico para estudiantes. Eres amable, directo y servicial. Usa tus herramientas para responder las preguntas."),
        ("human", "{input}"),
        ("placeholder", "{agent_scratchpad}"),
    ])
    
    # Creamos el agente uniendo el LLM, las herramientas y el prompt
    agent = create_tool_calling_agent(llm, tools, prompt)
    
    # El Executor es el que realmente corre el agente y maneja los ciclos de pensamiento
    agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True) # verbose=True para ver los pensamientos del agente
    
    return agent_executor